In [6]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import *

from qiskit_aer import Aer

We can simulate without using `qiskit` just by using python packages. There we don't need to worry about the initial state $\rho$. We assume it is given to us, in classical matrix form. So the expectation values can be calculated just by calculating traces.

# Notes
Let, $\rho = \sum_{i=1}^d c_i X_i$

It is assumed that $X_i$'s are independent and orthonormal i.e $Tr(X_i^\dagger X_j) = \delta_{ij}$.

Then, $Tr(\rho X_i) = c_i$

Our initial state is $\rho = \rho_1 \oplus \rho_2 \oplus \rho_3$, where,

$$\rho_1 = \frac{1}{3}
\begin{bmatrix}
    1 & 1 & 1 \\
    1 & 1 & 1 \\
    1 & 1 & 1
\end{bmatrix}
            \quad 
\rho_2 = \frac{1}{3}
\begin{bmatrix}
    1 & 1 & 1 \\
    1 & 1 & -1 \\
    1 & -1 & 1
\end{bmatrix} \quad
\rho_3 = \frac{1}{2}
\begin{bmatrix}
    1 & 0 \\ 0 & 1
\end{bmatrix}$$


We want to find `Pauli Decomposition` of $\rho$ i.e. to find $c_i$'s where $X_i$'s are Pauli matrices/tensor products of Pauli matrices.

# Reference from [here](https://docs.quantum.ibm.com/build/specify-observables-pauli)

In [7]:
#TODO: initialize a mixed state
from scipy.linalg import block_diag
from qiskit.quantum_info import DensityMatrix


# specify the quantum state in an array
rho_1 = (1/3)*np.array([[1,1,1],
                        [1,1,1],
                        [1,1,1]])

rho_2 = (1/3)*np.array([[1,1,1],
                        [1,1,-1],
                        [1,-1,1]])

rho_3 = (1/2)*np.array([[1,0],
                        [0,1]])

# direct sum of rho_1, rho_2, and rho_3

rho =  block_diag(rho_1, rho_2, rho_3)
# this is our initial mixed state

#convert rho into density matrix
rho = DensityMatrix(rho)

In [8]:
DensityMatrix.is_valid(rho)

False

In [9]:
# Pauli decomposition
from qiskit.quantum_info import SparsePauliOp

pauli_decomposition = SparsePauliOp.from_operator(rho)

In [10]:
# pauli decomposition
paulis = pauli_decomposition.to_list()

paulis

[('III', (0.375+0j)),
 ('IXI', (0.08333333333333333+0j)),
 ('IXX', (0.08333333333333333+0j)),
 ('IXZ', (0.08333333333333333+0j)),
 ('IYY', (0.08333333333333333-0j)),
 ('IZI', (-0.04166666666666666+0j)),
 ('XXI', (0.08333333333333333+0j)),
 ('XXX', (0.08333333333333333+0j)),
 ('XXZ', (-0.08333333333333333+0j)),
 ('XYY', (-0.08333333333333333+0j)),
 ('YXY', (0.08333333333333333-0j)),
 ('YYI', (0.08333333333333333-0j)),
 ('YYX', (0.08333333333333333-0j)),
 ('YYZ', (-0.08333333333333333+0j)),
 ('ZII', (-0.04166666666666667+0j)),
 ('ZIX', (0.16666666666666666+0j)),
 ('ZXI', (0.08333333333333333+0j)),
 ('ZXX', (0.08333333333333333+0j)),
 ('ZXZ', (0.08333333333333333+0j)),
 ('ZYY', (0.08333333333333333-0j)),
 ('ZZI', (0.04166666666666667+0j)),
 ('ZZX', (0.16666666666666666+0j))]

## Here though we have decomposed our state $\rho$ into Pauli matrices, we don't know how to initialize our circuit in this particular state 🥲

# Let us also try implementing this _from scratch_ without using `qiskit`.

In [1]:
import numpy as np
import sympy as sp

In [52]:
import numpy as np

def is_density_matrix(matrix):
    # Check if the matrix is square
    if matrix.shape[0] != matrix.shape[1]:
        print("Matrix is not square")
        return False
    
    # Check if the matrix is Hermitian
    if not np.allclose(matrix, matrix.conj().T):
        print("Matrix is not Hermitian")
        return False
    
    # Check if the trace of the matrix is 1
    if not np.isclose(np.trace(matrix), 1):
        print("Trace of the matrix is not 1")
        return False
    
    # Check if all eigenvalues are non-negative
    eigenvalues = np.linalg.eigvalsh(matrix)
    if not np.all(eigenvalues >= 0):
        print("Not all eigenvalues are non-negative")
        return False
    
    return True

In [64]:
# import numpy as np

# Generate a random Hermitian matrix
random_matrix = np.random.rand(2, 2) + 1j * np.random.rand(2, 2)
hermitian_matrix = (random_matrix + random_matrix.conj().T) / 2

# Normalize the matrix to have trace 1
trace = np.trace(hermitian_matrix)
density_matrix = hermitian_matrix / trace

print(density_matrix)   
is_density_matrix(density_matrix)

[[0.47745114+0.j         0.20956461-0.08448192j]
 [0.20956461+0.08448192j 0.52254886+0.j        ]]


True

In [107]:
from qutip import *
dm = rand_dm(5)
print(dm)
is_density_matrix(dm.full())
  

Quantum object: dims=[[5], [5]], shape=(5, 5), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 0.28474082-5.92230114e-19j  0.0332165 -7.12479998e-02j
   0.0554261 +7.19016757e-02j -0.06524217-1.31167879e-01j
  -0.09609442+6.94133485e-02j]
 [ 0.0332165 +7.12479998e-02j  0.19222892-3.99815356e-19j
  -0.04695128-5.15906989e-02j  0.08196304+2.84208391e-02j
  -0.03516011-5.36887955e-02j]
 [ 0.0554261 -7.19016757e-02j -0.04695128+5.15906989e-02j
   0.10927596-2.27282168e-19j -0.08223806-3.98941419e-02j
   0.00334973+7.93835268e-02j]
 [-0.06524217+1.31167879e-01j  0.08196304-2.84208391e-02j
  -0.08223806+3.98941419e-02j  0.24352848-5.06512909e-19j
  -0.01859696-5.15179286e-02j]
 [-0.09609442-6.94133485e-02j -0.03516011+5.36887955e-02j
   0.00334973-7.93835268e-02j -0.01859696+5.15179286e-02j
   0.17022582+1.72584055e-18j]]


True

In [110]:
I, X, Y, Z = qeye(2), sigmax(), sigmay(), sigmaz()
I, X, Y, Z = np.array(I), np.array(X), np.array(Y), np.array(Z)